In [ ]:
!nvidia-smi

Sat Nov 30 03:07:00 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Get Images + Metadata

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
%cd drive/MyDrive/T4SG St Jude/Labelled Assets/
!ls


/content/drive/.shortcut-targets-by-id/1OzE9n_wYhYcpK_L5s-1Tx1zZAOXd_f78/T4SG St Jude/Labelled Assets
 00047369-005_color_TIF.jpg	  00206983-014_TIF.jpg		  00216097-013-V2_TIF.jpg
 00154810-112_wm_TIF.jpg	  00207400-088_TIF.jpg		  00216097-014-V2_TIF.jpg
 00174912-082_wm_TIF.jpg	  00207400-092_TIF.jpg		  00216097-020-V2_TIF.jpg
 00183960-067_wm_TIF.jpg	  00207431-003_Wu_TIF.jpg	  00216182-019_TIF.jpg
 00191517-007_wm_JPG.jpg	  00207528-008_TIF.jpg		  00216191-008_TIF.jpg
 00194570-013_TIF.jpg		  00207528-019_TIF.jpg		  00216191-028_TIF.jpg
 00195485-003_TIF.jpg		  00207603-009_TIF.jpg		  00216322-011_TIF.jpg
 00195485-012_TIF.jpg		  00207610-013_TIF.jpg		  00216323-021_TIF.jpg
 00198073-006_TIF.jpg		  00207635-002_TIF.jpg		  00216338-005_TIF.jpg
 00198151-011_TIF.jpg		  00208305-011_TIF.jpg		  00216408-007_TIF.jpg
 00198973-026_JPG.jpg		  00208305-013_TIF.jpg		  00216511-003-V1_TIF.jpg
 00199428-025_TIF.jpg		  00208436-029_TIF.jpg		  00216511-010-V1_TIF.jpg
 00201051-092_TIF.jpg		  002

In [ ]:
! pip install exifread

import os
import exifread
from tqdm import tqdm

sample_images = []

missing_count = 0
for filepath_i in tqdm(os.listdir(os.getcwd())):
  # Open image file for reading (must be in binary mode)
  if not filepath_i.endswith(".jpg"):
    continue
  f = open(filepath_i, 'rb')

  # Return Exif tags
  tags = exifread.process_file(f)

  try:
    actual_labels = {
        "description" : tags["Image ImageDescription"]
    }

    sample_images.append((filepath_i, actual_labels))
  except:
    actual_labels = {
        "description" : "NOT AVAILABLE"
    }
    sample_images.append((filepath_i, actual_labels))
    missing_count += 1

print(f"Recovered {len(sample_images)}, Missing labels for {missing_count}")

100%|██████████| 203/203 [00:04<00:00, 42.41it/s] 

Recovered 199, Missing labels for 24


# Try Image-Captioning Models

There are a variety of pre-trained models which take in an image and a text-prompt and output a caption. We'll try seeing how well these do on our sample images without any fine-tuning.

- [Paligemma](https://huggingface.co/google/paligemma-3b-pt-448)
- [BLIP](https://github.com/salesforce/BLIP?tab=readme-ov-file)


[There's a lot of metadata we hope to predict, but the sample images don't currently have those.](https://docs.google.com/spreadsheets/d/1TWElrWaR4sm3CcFMju4zlzgFmKe5Uem1/edit?usp=sharing&ouid=115752792879348590018&rtpof=true&sd=true).

Let's start by predicting the "description," which looks to be a list of words describing the image, and "categories," which is not in the sample images but we can try first.

In [ ]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

output = []

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to("cuda")
cnt = 0

for filepath_i, actual_i in sample_images:
  # Read filepath_i, actual_i
  cnt += 1
  print(filepath_i, actual_i, cnt)

  # TODO: PROCESS THE IMAGE AND SEE THE DESCRIPTION
  raw_image = Image.open(filepath_i).convert('RGB')
  # Test 1: "building, light, location, " Encouragement works
  # Test 2: "building, light, location, campus, " More words more improvement but less original tokens
  # Test 3: "ysabp, kqvf, building, light, " Quite good but less tokens than ideal
  # Test 4: "a, b, c, d, e, light, " The front a, b, c, d, e seem to do nothing for list encouragement: sentences can be seen
  prompt = ""
  inputs = processor(raw_image, prompt, return_tensors="pt").to("cuda")
  out = model.generate(**inputs)
  promptless_output = processor.decode(out[0], skip_special_tokens=True)

  center_prompt = "in the center there is"
  inputs = processor(raw_image, center_prompt, return_tensors="pt").to("cuda")
  out = model.generate(**inputs, max_new_tokens=40)
  center_output = processor.decode(out[0], skip_special_tokens=True)

  corner_prompt = "in the corners there are"
  inputs = processor(raw_image, corner_prompt, return_tensors="pt").to("cuda")
  out = model.generate(**inputs, max_new_tokens=40)
  corner_output = processor.decode(out[0], skip_special_tokens=True)

  combined_output = promptless_output + center_output.replace(center_prompt, "")+ corner_output.replace(corner_prompt, "")

  print(combined_output)

  # print(promptless_output)

  # STORE THE IMAGE, THE ACTUAL DESCRIPTION, AND THE PREDICTED DESCRIPTION
  output.append({
      "image_path" : filepath_i,
      "actual": actual_i,
      "predicted" : combined_output, #TODO ADD SOMETHING HERE
  })

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

00206969-018_Kalodimos_TIF.jpg {'description': (0x010E) ASCII=00206969-018, 05-23-19, staff location portraits, individuals, researchers, Bruker Ascend 1.1 GHz magnet, NMR, man, male, Babis Kalodimos PhD (Structural Biology), @ 200} 1


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


arafed man in a suit reading a newspaper in a room a man reading a newspaper in a room a man in a suit reading a newspaper
00214636-006_TIF.jpg {'description': (0x010E) ASCII=00214636-006, 07-01-21, campus aerial, campus exterior, Hope garden, drone @ 188} 2
aerial view of a garden with a heart shaped maze and a sign that says hope a maze with a heart in the middle a maze with a heart in the middle
00204697-010_TIF.jpg {'description': (0x010E) ASCII=00204697-010, 01-31-19, research instruments, laboratory, reagent bottles, glassware, cleaning, autoclave, @ 200} 3
bottles and glasses are lined up on a conveyor belt a row of empty soda cans many empty glass bottles on the shelf
00206966-004_Abraham_TIF.jpg {'description': (0x010E) ASCII=00206966-004, 05-23-19, staff - location portraits, office, Computational Biology, male, Brian Abraham, @ 200} 4
there is a man sitting at a table with a laptop a man in a suit sitting at a table with a laptop a man in a suit sitting at a table with a lap

## Output Results in a CSV

In [ ]:
import pandas as pd
df = pd.DataFrame.from_dict(output)
df.to_csv("BLIPFullResults.csv")

# /usr/local/lib/python3.10/dist-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (96000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack. warnings.warn(

NameError: name 'output' is not defined